In [ ]:
import csv
import pandas as pd
import numpy as np
import sys
import warnings
import matplotlib.pyplot as plt
import statsmodels.regression.linear_model as lm
import statsmodels.api as sm

from scipy import linalg
from sklearn import datasets
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import StratifiedKFold, KFold, GridSearchCV
from sklearn.linear_model import LassoCV, LassoLarsCV, LassoLarsIC, RandomizedLasso, lasso_stability_path, ElasticNet, Lasso, Ridge
from sklearn.feature_selection import f_regression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import auc, precision_recall_curve, mean_squared_error
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.utils.extmath import pinvh
from sklearn.exceptions import ConvergenceWarning
from zipfile import ZipFile
from datetime import datetime

In [ ]:
def fillMissing(inputcsv, outputcsv):    
    # read input csv - takes time
    df = pd.read_csv(inputcsv, low_memory=False)
    # Fix date bug
    df.cf4fint = ((pd.to_datetime(df.cf4fint) - pd.to_datetime('1960-01-01')) / np.timedelta64(1, 'D')).astype(int)

    # replace NA's with median
    med2 = df.median()
    dfi = df.fillna(value=med2)
    
    #remove object columns
    dfn= dfi.select_dtypes(['number'])

    #remove NA columns
    a = dfn.notnull().all()
    dfn = dfn[a[a].index]

    # replace negative values with 1
    num = dfn._get_numeric_data()
    num[num < 0] = 1
    # write filled outputcsv
    dfn.to_csv(outputcsv, index=False)

In [ ]:
fillMissing('background.csv', 'output.csv')

In [ ]:
background = pd.read_csv("output.csv", low_memory=False)
background.sort_values(by='challengeID', inplace=True)
prediction = pd.read_csv("prediction_old.csv", low_memory=False)

In [ ]:
def filter_data(background):
	nRow = len(background) 
	nCol = len(background.iloc[0,:])

	X_all = background.drop(['challengeID', 'idnum'], axis=1)
	X_train = X_all.iloc[:2121,:]

	Y_train = pd.read_csv("train.csv", low_memory=False)
	return X_train, Y_train

In [ ]:
X_all,y_all = filter_data(background)

In [ ]:
def get_data_for_characteristic(X_train, Y_train, characteristic):
	all_char = Y_train[characteristic] #This is a Series

	#Remove rows where grit is NA
	char_defined = np.where(all_char.notnull())
	char = all_char.iloc[char_defined]
	X_train_char = X_train.iloc[char_defined]

	return X_train_char, char

In [ ]:
X_evict, y_evict = get_data_for_characteristic(X_all, y_all, 'eviction')
print len(X_evict.astype(float))
print len(X_evict.iloc[0, ])
print len(y_evict.astype(float))

In [ ]:
gaussian_links = sm.families.family.Gaussian.links

In [ ]:
binomial_links = sm.families.family.Binomial.links